## **DAY 8-Unity Catalog Governance**

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS workspace;

USE CATALOG workspace;

CREATE SCHEMA IF NOT EXISTS ecommerce;
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;



In [0]:
%sql
CREATE TABLE workspace.bronze.events
USING DELTA
AS
SELECT *
FROM read_files(
  '/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv',
  format => 'csv',
  header => true
);



num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE TABLE workspace.silver.events
USING DELTA
AS
SELECT *
FROM workspace.bronze.events
WHERE event_time IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE TABLE workspace.gold.daily_events
USING DELTA
AS
SELECT
  DATE(event_time) AS event_date,
  COUNT(*) AS total_events
FROM workspace.silver.events
GROUP BY DATE(event_time);



num_affected_rows,num_inserted_rows


In [0]:
%sql
GRANT SELECT ON SCHEMA workspace.bronze TO `account users`;
GRANT SELECT ON SCHEMA workspace.silver TO `account users`;
GRANT SELECT ON SCHEMA workspace.gold TO `account users`;





In [0]:
%sql
CREATE VIEW workspace.gold.events_view AS
SELECT event_date, total_events
FROM workspace.gold.daily_events;


In [0]:
%sql
GRANT SELECT ON VIEW workspace.gold.events_view TO `account users`;



In [0]:
%sql
SHOW TABLES IN workspace.bronze;
SHOW TABLES IN workspace.silver;
SHOW TABLES IN workspace.gold;


database,tableName,isTemporary
gold,daily_events,false
gold,events_view,false
